In [1]:
from __future__ import print_function
from __future__ import print_function
from __future__ import division
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import matplotlib.pyplot as plt
import csv
from torchvision import models
import time
import os
import copy
import cv2
import numpy as np
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import scipy.io
import numpy as np
from pathlib import Path
import glob

import torch.utils.data as data
import pandas as pd
from PIL import Image
import os.path

PyTorch Version:  2.1.2+cu121
Torchvision Version:  0.16.2+cu121


In [2]:
IMG_EXTENSIONS = [
   '.jpg', '.JPG', '.jpeg', '.JPEG',
   '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP','.mat',
]


def is_image_file(filename):
   return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
   classes = os.listdir(dir)
   classes.sort()
   class_to_idx = {classes[i]: i for i in range(len(classes))}
   return classes, class_to_idx


def make_dataset(dir, class_to_idx):
   images = []
   for target in os.listdir(dir):
       d = os.path.join(dir, target)
       if not os.path.isdir(d):
           continue

       for filename in os.listdir(d):
           if is_image_file(filename):
               path = '{0}/{1}'.format(target, filename)
               #print(path)
               item = (path, class_to_idx[target])
               images.append(item)

   return images

def default_loader(path):
   return Image.open(path).convert('RGB')

def mat_loader(path):
   return scipy.io.loadmat(path)

In [3]:
classes1, class_to_idx1 = find_classes("/home/user1/icip/similar/train/")
       
imgs1 = make_dataset("/home/user1/icip/random/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/similar/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/merged/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_Note_8Pro': 7, 'D09_Samsung_Galaxy_J8_10G': 8, 'D10_Samsung_Galaxy_F41': 9, 'D11_OnePlus_8T': 10, 'D12_Vivo_Y02t': 11, 'D13_Oppo_A17k': 12, 'D14_Samsung_Galaxy_S20FE': 13, 'D15_Motorola_Motog60': 14, 'D16_Samsung_Galaxy_S21FE': 15, 'D17_Apple_Iphone_12': 16, 'D18_IQOO_Z3': 17, 'D19_IQOO_Z6_Lite': 18, 'D20_Motorola_MotoG73_5G': 19, 'D21_OnePlus_10Pro_5G': 20, 'D22_Poco_F5': 21, 'D23_Poco_F5_Pro_5G': 22, 'D24_Realme_8': 23, 'D25_Realme_X3_Superzoom': 24, 'D26_Redmi_9i_Sport': 25, 'D27_Redmi_Note10_Pro': 26, 'D28_Apple_Iphone_13': 27, 'D29_Apple_Iphone_15': 28, 'D30_Vivo_Y75': 29} 1071
{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_Note

In [4]:
class ImageFolderLoader(data.Dataset):
   def __init__(self, root1,transform_1=None,
                target_transform=None,
                loader=default_loader):
       classes1, class_to_idx1 = find_classes(root1)
       
       imgs1 = make_dataset(root1, class_to_idx1)
      

       self.root1 = root1
       self.imgs1 = imgs1
       self.classes1 = classes1
       self.class_to_idx1 = class_to_idx1
       self.transform_1 = transform_1
       self.target_transform = target_transform
       self.loader = loader
        
       self.img_noise = None
       self.img_rgb = None
    
    
   def __getitem__(self, index):
    

       path1, target1 = self.imgs1[index]    
       filename = Path(path1).stem 
       img1 = self.loader(os.path.join(self.root1, path1))  
    
       self.img_rgb = img1
       

       if self.transform_1 is not None:
           img1 = self.transform_1(self.img_rgb)
        
       if self.target_transform is not None:
           target1 = self.target_transform(target)
            
       
       return img1,target1

   def __len__(self):
       return len(self.imgs1)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        resnet1 = models.resnet50(pretrained=True)
        modules1 = list(resnet1.children())[:-1]      # delete the last fc layer.
        self.resnet1 = nn.Sequential(*modules1)

    
    
        self.fc1 = nn.Linear(2048, 2048)
        self.fc2 = nn.Linear(2048,30)
        
        
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
       

    def forward(self, x):
        
        x = self.resnet1(x)
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.fc2(x)
       
        out_fc = x
        output = self.logsoftmax(x)
        
        return output, out_fc
    

In [6]:
data_transforms = transforms.Compose([
transforms.Resize((256,256)),
transforms.ToTensor()
])


batchsize=1

val_dataset_random = ImageFolderLoader(
        "/home/user1/icip/random/test/",
        transform_1=data_transforms
    )

test_loader_random = torch.utils.data.DataLoader(
        val_dataset_random, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_similar = ImageFolderLoader(
        "/home/user1/icip/similar/test/",
        transform_1=data_transforms
    )

test_loader_similar = torch.utils.data.DataLoader(
        val_dataset_similar, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_merged = ImageFolderLoader(
        "/home/user1/icip/merged/test/",
        transform_1=data_transforms
    )

test_loader_merged = torch.utils.data.DataLoader(
        val_dataset_merged, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

In [7]:
len(val_dataset_random), len(val_dataset_similar), len(val_dataset_merged)

(1071, 960, 2031)

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0')
model = Net().to(device)

/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model = torch.load("Chen_trained_model_similar_final")
model.eval()
criterion = nn.NLLLoss()

test_loss = 0
total_loss = 0
train_loss = 0
correct = 0
test_epoch = 1
total_correct_test = 0
total_correct_train = 0
total_correct = 0
epoch = 0

with torch.no_grad():
    for batch_idx, (imgs1,labels1) in enumerate(test_loader_random):
        
        img_org,target = imgs1.to(device),labels1.to(device)
        output, fc_feature = model(img_org)
        loss = criterion(output, target)

        test_loss = test_loss + loss.item()  # sum up batch loss

        _, predicted = torch.max(output.data, 1)
        total = float(len(target))
        correct = (predicted == target).sum()
        total_correct_test = total_correct_test + np.float64(correct.cpu().numpy())


        if batch_idx % 200 == 0:
            print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img_org), len(test_loader_random.dataset),
                100. * batch_idx / len(test_loader_random), loss.item()))

        if batch_idx % 200 == 0:
            accuracy = 100.0 * float(correct) / total
            print("Accuracy===",accuracy)


alpha = (len(test_loader_random.dataset))/ batchsize
print(test_loss)
test_loss /= alpha

ILA_random=np.around((100* total_correct_test / len(test_loader_random.dataset)),decimals=2)


print('\nImage Level Accuracy: {}%\n'.format(ILA_random))

Test Epoch: 0 [0/1071 (0%)]	Loss: 6.729555
Accuracy=== 0.0
Test Epoch: 0 [200/1071 (19%)]	Loss: 0.235904
Accuracy=== 100.0
Test Epoch: 0 [400/1071 (37%)]	Loss: 0.000349
Accuracy=== 100.0
Test Epoch: 0 [600/1071 (56%)]	Loss: 6.979759
Accuracy=== 0.0
Test Epoch: 0 [800/1071 (75%)]	Loss: 0.127837
Accuracy=== 100.0
Test Epoch: 0 [1000/1071 (93%)]	Loss: 8.957326
Accuracy=== 0.0
6024.908671816747

Image Level Accuracy: 32.96%



In [10]:
test_loss = 0
total_loss = 0
train_loss = 0
correct = 0
test_epoch = 1
total_correct_test = 0
total_correct_train = 0
total_correct = 0
epoch = 0

with torch.no_grad():
    for batch_idx, (imgs1,labels1) in enumerate(test_loader_similar):
        
        img_org,target = imgs1.to(device),labels1.to(device)
        output, fc_feature = model(img_org)
        loss = criterion(output, target)

        test_loss = test_loss + loss.item()  # sum up batch loss

        _, predicted = torch.max(output.data, 1)
        total = float(len(target))
        correct = (predicted == target).sum()
        total_correct_test = total_correct_test + np.float64(correct.cpu().numpy())


        if batch_idx % 200 == 0:
            print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img_org), len(test_loader_similar.dataset),
                100. * batch_idx / len(test_loader_similar), loss.item()))

        if batch_idx % 200 == 0:
            accuracy = 100.0 * float(correct) / total
            print("Accuracy===",accuracy)


alpha = (len(test_loader_similar.dataset))/ batchsize
print(test_loss)
test_loss /= alpha

ILA_similar=np.around((100* total_correct_test / len(test_loader_similar.dataset)),decimals=2)


print('\nImage Level Accuracy: {}%\n'.format(ILA_similar))

Test Epoch: 0 [0/960 (0%)]	Loss: 0.943109
Accuracy=== 0.0
Test Epoch: 0 [200/960 (21%)]	Loss: 0.000003
Accuracy=== 100.0
Test Epoch: 0 [400/960 (42%)]	Loss: 0.007061
Accuracy=== 100.0
Test Epoch: 0 [600/960 (62%)]	Loss: 0.000000
Accuracy=== 100.0
Test Epoch: 0 [800/960 (83%)]	Loss: 0.000021
Accuracy=== 100.0
1301.175138946286

Image Level Accuracy: 80.1%



In [11]:
test_loss = 0
total_loss = 0
train_loss = 0
correct = 0
test_epoch = 1
total_correct_test = 0
total_correct_train = 0
total_correct = 0
epoch = 0

with torch.no_grad():
    for batch_idx, (imgs1,labels1) in enumerate(test_loader_merged):
        
        img_org,target = imgs1.to(device),labels1.to(device)
        output, fc_feature = model(img_org)
        loss = criterion(output, target)

        test_loss = test_loss + loss.item()  # sum up batch loss

        _, predicted = torch.max(output.data, 1)
        total = float(len(target))
        correct = (predicted == target).sum()
        total_correct_test = total_correct_test + np.float64(correct.cpu().numpy())


        if batch_idx % 200 == 0:
            print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img_org), len(test_loader_merged.dataset),
                100. * batch_idx / len(test_loader_merged), loss.item()))

        if batch_idx % 200 == 0:
            accuracy = 100.0 * float(correct) / total
            print("Accuracy===",accuracy)


alpha = (len(test_loader_merged.dataset))/ batchsize
print(test_loss)
test_loss /= alpha

ILA_merged=np.around((100* total_correct_test / len(test_loader_merged.dataset)),decimals=2)


print('\nImage Level Accuracy: {}%\n'.format(ILA_merged))

Test Epoch: 0 [0/2031 (0%)]	Loss: 0.943109
Accuracy=== 0.0
Test Epoch: 0 [200/2031 (10%)]	Loss: 0.000104
Accuracy=== 100.0
Test Epoch: 0 [400/2031 (20%)]	Loss: 0.000029
Accuracy=== 100.0
Test Epoch: 0 [600/2031 (30%)]	Loss: 8.885495
Accuracy=== 0.0
Test Epoch: 0 [800/2031 (39%)]	Loss: 0.123875
Accuracy=== 100.0
Test Epoch: 0 [1000/2031 (49%)]	Loss: 7.128145
Accuracy=== 0.0
Test Epoch: 0 [1200/2031 (59%)]	Loss: 0.001118
Accuracy=== 100.0
Test Epoch: 0 [1400/2031 (69%)]	Loss: 0.000045
Accuracy=== 100.0
Test Epoch: 0 [1600/2031 (79%)]	Loss: 0.000000
Accuracy=== 100.0
Test Epoch: 0 [1800/2031 (89%)]	Loss: 1.793527
Accuracy=== 0.0
Test Epoch: 0 [2000/2031 (98%)]	Loss: 0.000000
Accuracy=== 100.0
7326.083810763034

Image Level Accuracy: 55.24%



In [12]:
print(f"Image Level Accuracy (Trained on Random Set, Tested on Random Set): {ILA_random}%")
print(f"Image Level Accuracy (Trained on Random Set, Tested on Similar Set): {ILA_similar}%")
print(f"Image Level Accuracy (Trained on Random Set, Tested on Merged Set): {ILA_merged}%")

Image Level Accuracy (Trained on Random Set, Tested on Random Set): 32.96%
Image Level Accuracy (Trained on Random Set, Tested on Similar Set): 80.1%
Image Level Accuracy (Trained on Random Set, Tested on Merged Set): 55.24%
